<a href="https://colab.research.google.com/github/olstjr/Allora_Chatbot_in_THU/blob/main/%EC%A0%84%EC%A3%BC%EB%8C%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 13.9 MB/s eta 0:00:00


In [ ]:
!playwright install


163.5 MiB [] 0% 0.0s163.5 MiB [] 0% 60.5s163.5 MiB [] 0% 34.5s163.5 MiB [] 0% 22.9s163.5 MiB [] 0% 13.8s163.5 MiB [] 0% 9.3s163.5 MiB [] 1% 8.5s163.5 MiB [] 1% 7.7s163.5 MiB [] 1% 6.9s163.5 MiB [] 2% 6.3s163.5 MiB [] 2% 6.1s163.5 MiB [] 3% 5.6s163.5 MiB [] 3% 5.3s163.5 MiB [] 4% 5.1s163.5 MiB [] 4% 5.0s163.5 MiB [] 5% 5.0s163.5 MiB [] 5% 4.9s163.5 MiB [] 6% 4.9s163.5 MiB [] 6% 5.0s163.5 MiB [] 6% 5.2s163.5 MiB [] 7% 5.2s163.5 MiB [] 7% 5.1s163.5 MiB [] 7% 5.0s163.5 MiB [] 8% 5.0s163.5 MiB [] 8% 4.9s163.5 MiB [] 9% 5.0s163.5 MiB [] 9% 5.1s163.5 MiB [] 10% 5.2s163.5 MiB [] 10% 5.4s163.5 MiB [] 11% 5.4s163.5 MiB [] 11% 5.3s163.5 MiB [] 11% 5.4s163.5 MiB [] 11% 5.3s163.5 MiB [] 12% 5.2s163.5 MiB [] 12% 5.1s163.5 MiB [] 13% 5.1s163.5 MiB [] 13% 5.0s163.5 MiB [] 13% 5.1s163.5 MiB [] 13% 5.2s163.5 MiB [] 14% 5.1s163.5 MiB [] 15% 5.1s163.5 MiB [] 16% 5.0s163.5 MiB [] 16% 4.9s163.5 MiB [] 17% 4.8s163.5 MiB [] 17% 4.7s163.5 MiB [] 18% 4.7s163.5 MiB [] 18% 4.6s163.5 MiB [] 19% 4.5s163.5 MiB [] 20

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup

# Jupyter Notebook에서 실행 가능하도록 설정
nest_asyncio.apply()

async def fetch_jju_scholarship_notices():
    base_url = "https://www.jj.ac.kr/jj/community/notice/janghaknotice.jsp?mode=list&board_no=1077&pager.offset={}"
    notices = []
    page_offsets = range(0, 41, 20)  # 최대 3페이지까지 탐색

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # ✅ 디버깅을 위해 headless=False로 변경
        context = await browser.new_context()

        # ✅ 공지사항 목록 페이지에서 세션 유지
        page = await context.new_page()
        await page.goto(base_url.format(0), wait_until="networkidle")

        for offset in page_offsets:
            list_page_url = base_url.format(offset)
            await page.goto(list_page_url, wait_until="networkidle")

            soup = BeautifulSoup(await page.content(), "html.parser")

            # 🔹 공지사항 제목 및 상세 URL 추출
            notice_items = soup.select("tr td:nth-child(2) a")
            notice_dates = soup.select("tr td:nth-child(3)")

            if not notice_items:  # 데이터가 없으면 종료
                break

            for item, date_tag in zip(notice_items, notice_dates):
                title = item.text.strip()
                date_str = date_tag.text.strip()

                # ✅ 링크를 직접 방문하는 것이 아니라 클릭 방식으로 이동하도록 설정
                element_handle = await page.query_selector(f'text="{title}"')
                if element_handle:
                    print(f"🔹 클릭 시도: {title}")
                    await element_handle.click()  # ✅ 클릭 방식으로 이동

                    # ✅ 페이지 로드 완료까지 대기
                    await page.wait_for_load_state("networkidle")

                    # ✅ 리디렉션 감지: 만약 index.jsp로 이동하면 다시 시도
                    if "index.jsp" in page.url:
                        print(f"⚠️ {title} 페이지가 index.jsp로 리디렉션됨. 크롤링 불가")
                        await page.go_back()  # 목록 페이지로 다시 이동
                        continue

                    final_soup = BeautifulSoup(await page.content(), "html.parser")

                    # 🔹 상세 내용 크롤링
                    content_div = final_soup.select_one("div:nth-of-type(1) > div:nth-of-type(2) > div:nth-of-type(2) > div:nth-of-type(3) > div > div:nth-of-type(1) > div > div:nth-of-type(3) > div")

                    if content_div:
                        content = content_div.get_text("\n", strip=True)
                    else:
                        content = "본문 없음"

                    # 🔹 "장학"이 포함된 공지만 저장
                    if "장학" in title.lower() or "장학" in content.lower():
                        notices.append({
                            "title": title,
                            "url": page.url,  # ✅ 현재 페이지 URL 저장
                            "date": date_str,
                            "content": content
                        })

                    await page.go_back()  # ✅ 목록 페이지로 다시 돌아가기

        await browser.close()

    return notices

# 📌 크롤링 실행
notices_data = asyncio.run(fetch_jju_scholarship_notices())
print(f"총 {len(notices_data)}개의 장학 공지사항을 크롤링했습니다.\n")

# ✅ 크롤링된 데이터 출력 (최대 5개 미리보기)
for i, notice in enumerate(notices_data[:5], 1):
    content_preview = notice['content'][:300]  # 본문 앞 300자만 출력
    print(f"🔹 [{i}] {notice['title']} ({notice['date']})")
    print(f"   👉 {notice['url']}")
    print(f"   📄 {content_preview}...\n")


🔹 클릭 시도: 2025-1학기 교내장학금 신청안내(다문화, 장애인자녀, 장애대학생, 특수환경지원 장학금)
🔹 클릭 시도: 주거안정장학금 신청(2/4 화~3/18 화) 안내
🔹 클릭 시도: 2025-1학기 국가장학금 2차 신청기간(2/4 화~3/18 화) 안내
🔹 클릭 시도: 2024년 하반기 대학생 학자금 대출이자 지원 공고(울산광역시)
🔹 클릭 시도: 2025년 상반기 대학생 학자금 대출이자 지원 공고(경기도)


ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("div.fr-view") to be visible\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("div.fr-view") to be visible



🔹 클릭 시도: 2025학년도 1학기 국가근로장학금 (2차) 신청 안내(2. 4. 9시 ~ 3. 18. 18시)
🔹 클릭 시도: 2025년 상반기 학자금 대출이자 지원사업 홍보(제주특별자치도)
🔹 클릭 시도: 2025학년도 1학기 학자금 대출 업무처리기준 안내
🔹 클릭 시도: 2025학년도 학자금 지원 가능 대학 및 학자금 지원 제한 대학 안내
🔹 클릭 시도: 중소기업 취업연계 장학사업(희망사다리1유형) 의무종사 기업기준 안내
🔹 클릭 시도: '국가근로, 대학생 청소년교육지원사업은 휴강을 인정하지 않는다'는 말의 의미(휴보강 기간 주의)
🔹 클릭 시도: 국가보훈장학금 국고보조금 신청 안내
🔹 클릭 시도: ★교내외 장학금 수령 계좌 inSTAR 등록 방법 안내★
🔹 클릭 시도: (한국장학재단) 2025년 특별상환유예대출  홍보
🔹 클릭 시도: 해양수산부 극지연구 전문인력 양성 장학금 신규 장학생 선발 공고(~2. 3.)
🔹 클릭 시도: 2025-1학기 목회자 및 목회자 자녀 장학금 신청 안내
🔹 클릭 시도: 한국장학재단 글로벌 현장학습 사업 소개(파란사다리, 한일 대학생 연수)
🔹 클릭 시도: 2025년 서울특별시 서울런 멘토단 참여 신청 안내(~1. 13.)
🔹 클릭 시도: 2025년 1학기 한국장학재단 푸른등대 기부장학금 신청 안내(12.27.~25.1.20 18시)
🔹 클릭 시도: (신청기간 연장)2025년 1학기 한국농어촌희망재단 청년창업농장학금 신청 안내(~25. 1. 13. 17시까지)
🔹 클릭 시도: 제29회 (재)부산진구장학회 장학생 선발 계획 공고(1. 6.~1. 17.)
🔹 클릭 시도: 2024-2학기 국가근로장학금 신규장학생 운영 안내(대체선발, 동계방학집중근로)
🔹 클릭 시도: 2025년 (재)협성문화재단 협성장학생 선발 안내(12. 16.~2025. 1.6.)
🔹 클릭 시도: 한국서부발전과 함께하는 서부공감 위피스쿨 15기 멘토 모집(~12.11.)
🔹 클릭 시도: 2025년도 제1기 전주시 글로벌 인재양성  영어능력강화사업 장학생 선발 

In [ ]:
import os
from google.colab import files

save_path = "/content/jju_scholarship_200.json"

# ✅ 기존 파일 삭제 후 다시 저장
if os.path.exists(save_path):
    os.remove(save_path)
    print("🗑️ 기존 JSON 파일 삭제 완료")

# ✅ 최신 JSON 데이터 저장
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(notices_data, f, ensure_ascii=False, indent=4)

# 파일 다운로드
files.download(save_path)

print(f"✅ JSON 파일이 저장되었고 다운로드할 수 있습니다: {save_path}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ JSON 파일이 저장되었고 다운로드할 수 있습니다: /content/jnu_scholarship_200.json
